In [ ]:
#installing the requried libraries

%pip install nltk beautifulsoup4 requests

### Importing the Required libraries

In [3]:
# Import necessary libraries
import requests  # For sending HTTP requests to fetch webpage data
from bs4 import BeautifulSoup  # For parsing and extracting data from HTML

# Define the URL of the webpage to scrape
url = "https://en.wikipedia.org/wiki/Machine_learning"
filename = "ml.txt"
# Function to scrape and save webpage content
def scrape_page(url,filename):
    # Fetch raw data from the webpage
    website_raw_data = requests.get(url)
    
    # Parse the HTML content using BeautifulSoup
    website_data = BeautifulSoup(website_raw_data.content, 'html.parser')
    
    # Extract all paragraph tags from the parsed HTML
    paragraphs = website_data.find_all('p')

    # Open a file in write mode to save the content
    with open(filename, mode='w', encoding='utf-8') as file:
        # Write the text of each paragraph into the file
        for para in paragraphs:
            file.write(para.getText())

# Call the function with the defined URL
scrape_page(url,filename)


In [ ]:
# Importing the nltk library for natural language processing tasks
import nltk
# Downloading the 'stopwords' dataset used for filtering common words like 'the', 'is', etc.
nltk.download('stopwords')
# Importing the stopwords module from nltk.corpus
from nltk.corpus import stopwords

def Read_file(filename):
    # Opening the file with 'filename' in read mode with UTF-8 encoding
    # Reading its content into a variable called 'original_context'
    with open(filename, mode='r', encoding='utf-8') as file:
        original_context = file.read()
    return original_context


file_text = Read_file(filename)



1. **Importing NLTK**:
    - The `nltk` library is used for various natural language processing tasks.
    - `stopwords`: A corpus of common words (e.g., "the", "is") that are often removed during text processing.

2. **Downloading Stopwords**:
    - The `nltk.download('stopwords')` function ensures the stopwords dataset is available locally.


In [7]:
# Importing the word_tokenize function from the nltk library
from nltk import word_tokenize

# Tokenizing the text into individual words using word_tokenize
# Each word in the text is separated into a list element
file_text_wordtokens = word_tokenize(file_text)

# Printing the list of word tokens
print(file_text_wordtokens)

['Machine', 'learning', '(', 'ML', ')', 'is', 'a', 'field', 'of', 'study', 'in', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'development', 'and', 'study', 'of', 'statistical', 'algorithms', 'that', 'can', 'learn', 'from', 'data', 'and', 'generalize', 'to', 'unseen', 'data', ',', 'and', 'thus', 'perform', 'tasks', 'without', 'explicit', 'instructions', '.', '[', '1', ']', 'Advances', 'in', 'the', 'field', 'of', 'deep', 'learning', 'have', 'allowed', 'neural', 'networks', 'to', 'surpass', 'many', 'previous', 'approaches', 'in', 'performance', '.', '[', '2', ']', 'ML', 'finds', 'application', 'in', 'many', 'fields', ',', 'including', 'natural', 'language', 'processing', ',', 'computer', 'vision', ',', 'speech', 'recognition', ',', 'email', 'filtering', ',', 'agriculture', ',', 'and', 'medicine', '.', '[', '3', ']', '[', '4', ']', 'The', 'application', 'of', 'ML', 'to', 'business', 'problems', 'is', 'known', 'as', 'predictive', 'analytics', '.', 'Statistics', 'and', 'mathema

In [9]:
# Importing the 'punctuation' module from the string library
from string import punctuation

# Defining the set of punctuations and adding newline characters to it
punctuations = punctuation + '\n'

# Loading the list of English stopwords from the NLTK library
eng_stopword = stopwords.words('english')

# Initializing an empty list to store tokens without stopwords or punctuation
clean_word_tokens = []

# Iterating over each token in the word token list
for word in file_text_wordtokens:
    # Filtering out stopwords and punctuation by converting the word to lowercase
    if word.lower() not in eng_stopword and word.lower() not in punctuations:
        clean_word_tokens.append(word)  # Adding filtered words to the list


In [13]:
# Function to calculate word frequencies in a tokenized text
def get_word_frequencies(word_tokens):
    # Initialize an empty dictionary to store word frequencies
    words_frequencie = {}
    
    # Iterate through each word in the tokenized list
    for word in word_tokens:
        # If the word is not already a key in the dictionary, add it with a frequency of 1
        if word not in words_frequencie:
            words_frequencie[word] = 1
        else:
            # If the word is already a key, increment its frequency by 1
            words_frequencie[word] += 1
    
    # Return the dictionary containing word frequencies
    return words_frequencie

words_frequencie = get_word_frequencies(clean_word_tokens)

In [16]:
# Function to calculate normalized word weights based on their frequency
def get_words_weight(words_frequencie):
    # Determine the maximum frequency in the word frequencies dictionary
    max_frequency = max(words_frequencie.values())
    
    # Initialize an empty dictionary to store normalized weights
    words_weight = {}
    
    # Iterate through each word in the word frequencies dictionary
    for word in words_frequencie.keys():
        # Calculate the normalized weight for the word
        normalized_weight = words_frequencie[word] / max_frequency
        # Add the word and its weight to the word_weights dictionary
        words_weight[word] = normalized_weight
    
    # Return the dictionary containing normalized word weights
    return words_weight
words_weight = get_words_weight(words_frequencie)

In [17]:
print(words_weight)

{'Machine': 0.0797872340425532, 'learning': 1.0, 'ML': 0.02127659574468085, 'field': 0.10106382978723404, 'study': 0.0425531914893617, 'artificial': 0.12234042553191489, 'intelligence': 0.05319148936170213, 'concerned': 0.015957446808510637, 'development': 0.005319148936170213, 'statistical': 0.047872340425531915, 'algorithms': 0.2393617021276596, 'learn': 0.05319148936170213, 'data': 0.5212765957446809, 'generalize': 0.010638297872340425, 'unseen': 0.015957446808510637, 'thus': 0.026595744680851064, 'perform': 0.05851063829787234, 'tasks': 0.047872340425531915, 'without': 0.047872340425531915, 'explicit': 0.010638297872340425, 'instructions': 0.005319148936170213, '1': 0.015957446808510637, 'Advances': 0.005319148936170213, 'deep': 0.031914893617021274, 'allowed': 0.005319148936170213, 'neural': 0.10106382978723404, 'networks': 0.0797872340425532, 'surpass': 0.010638297872340425, 'many': 0.07446808510638298, 'previous': 0.026595744680851064, 'approaches': 0.05851063829787234, 'perform

In [20]:
# Importing the sent_tokenize function from the nltk library
from nltk import sent_tokenize

# Tokenizing the original text into sentences using sent_tokenize
# Each sentence in the text becomes an element in the list
file_text_sentences_tokens = sent_tokenize(file_text)

# Displaying the list of sentences from index 0 to 10
file_text_sentences_tokens[0:10]


['Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.',
 '[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.',
 '[2]\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.',
 '[3][4] The application of ML to business problems is known as predictive analytics.',
 'Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning.',
 'Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.',
 '[6][7]\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing mach

In [ ]:
# Function to calculate weights for each sentence based on word weights
def get_sentence_weight(sentence_tokens, word_weights):
    
    # Initialize an empty dictionary to store sentence weights
    sentence_weights = dict()

    # Iterating through each sentence in the tokenized sentences
    for line in sentence_tokens:
        # Tokenize the current sentence into words using word_tokenize
        line_word_tokens = word_tokenize(line)
        
        # Initialize the sentence weight to 0
        sentence_weight = 0.0
        
        # Calculate the cumulative weight of words in the sentence
        for word in line_word_tokens:
            # Check if the word exists in the word_weights dictionary
            if word in word_weights.keys():
                # Add the word's weight to the sentence's total weight
                sentence_weight += word_weights[word]
            else:
                # No change if the word is not in word_weights
                sentence_weight = sentence_weight
        
        # Add or update the sentence's weight in the sentence_weights dictionary
        if line in sentence_weights:
            sentence_weights[line] += sentence_weight
        else:
            sentence_weights[line] = sentence_weight

    # Return the dictionary containing the weight for each sentence
    return sentence_weights


# file_text_sentences_tokens is a list of tokenized sentences
# and words_weight is the dictionary containing word weights
sentence_weight = get_sentence_weight(file_text_sentences_tokens, words_weight)

# Printing the sentence weights
print(sentence_weight)


{'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.': 3.0904255319148937, '[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.': 1.574468085106383, '[2]\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.': 0.46808510638297873, '[3][4] The application of ML to business problems is known as predictive analytics.': 0.23404255319148937, 'Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning.': 1.8670212765957446, 'Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.': 1.9734042553191489, '[6][7]\nF

In [ ]:
from heapq import nlargest

# Function to generate a summary by selecting top sentences based on their weights
def get_summary(sentences_weights, lenofsummary=0.2):
    """
    This function generates a summary by selecting the top sentences 
    based on their weights.
    
    :param sentences_weights: Dictionary containing sentences as keys and their weights as values
    :param lenofsummary: Fraction of sentences to select for the summary (default is 0.2, which means 20%)
    :return: List of top sentences selected for the summary
    """
    
    # Calculate the number of sentences to select based on the specified fraction
    select_len = int(len(sentences_weights) * lenofsummary)
    
    # Use nlargest to get the top sentences based on their weights
    summary = nlargest(select_len, sentences_weights, key=sentences_weights.get)
    
    # Return the selected top sentences
    return summary


In [27]:
summary = get_summary(sentence_weight,lenofsummary=0.1)

In [28]:
new_summary = " ".join(summary)
with open('ml_summary.txt','w',encoding='utf-8') as f:
    f.write(new_summary)